In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import glob
import seaborn as sns
from experiment_utils import Experiment, ExperimentFilter, Plotting, IsstPerfProfile

In [ ]:
experiments = Experiment.get_experiments()
experiments = list(filter(ExperimentFilter.by_experiment_name('avx-frequency-license-bands'), experiments))
experiment = ExperimentFilter.get_latest(experiments)
experiment

In [ ]:
result = IsstPerfProfile.parse(f'{experiment.path}/isst-perf-profile.log')
result.profiles

In [ ]:
print(result.profiles[0].turbo_levels)

In [ ]:
license_level_to_name = {
    '0': 'SSE',
    '1': 'AVX2',
    '2': 'AVX512',
    '3': 'AMX'
}

LIC_NAME = 'License name'
CORE_COUNT = 'Number of active cores'
MAX_TURBO_FREQ = 'Maximum opportunistic turbo frequency [MHz]'

freq_vs_number_of_cores = pd.DataFrame(result.profiles[0].turbo_levels)
freq_vs_number_of_cores[LIC_NAME] = freq_vs_number_of_cores['level'].apply(lambda v: license_level_to_name[v])
freq_vs_number_of_cores[CORE_COUNT] = freq_vs_number_of_cores['core_count']
freq_vs_number_of_cores[MAX_TURBO_FREQ] = freq_vs_number_of_cores['max_turbo_frequency_mhz']

freq_vs_number_of_cores


In [ ]:
hue_order=['AMX', 'AVX512', 'AVX2', 'SSE']

In [ ]:
# plt.figure(figsize=(10,6))
ax = sns.scatterplot(data=freq_vs_number_of_cores, x=CORE_COUNT, y=MAX_TURBO_FREQ, hue=LIC_NAME, linewidth=1.5, marker='o', hue_order=hue_order)

# Overlay the red markers for the SSE2 license on the left
sse_marker = freq_vs_number_of_cores[freq_vs_number_of_cores[LIC_NAME] == 'SSE']
plt.plot(sse_marker[CORE_COUNT], sse_marker[MAX_TURBO_FREQ], color=sns.color_palette()[3], linestyle='None', markeredgewidth=0.0, ms=4.5, marker=matplotlib.markers.MarkerStyle("o", fillstyle="left"))

buckets = [28,34,40,44,48,50,54,56]

for i in buckets:
    plt.axvline(x=i, color='black', dashes=[2], linewidth=1)

# ax.set_ylim(0, 4000)
ax.set_xlim(0, 58)
ax.set_xticks([0] + buckets)

plt.xticks(rotation=45)
ax.tick_params(axis='x', pad=0)

Plotting.savefig(experiment, 'avx-frequency-license-bands.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)